In [ ]:
import numpy as np
import cv2
import pylab as pl
import csv
import matplotlib.pyplot as plt
import math

def box_counting(image, box_sizes):
	box_counts = []

	for size in box_sizes:
		count = 0

		for i in range(0, image.shape[0], size):
			for j in range(0, image.shape[1], size):
				unique_intensities = np.unique(image[i:i+size, j:j+size])
				count += math.ceil(np.max(unique_intensities) / size)
				
		box_counts.append(count)

	return box_counts

def calculate_fractal_dimension(box_counts, box_sizes):
	log_sizes = np.log(box_sizes)
	log_counts = np.log(box_counts)

	m, c = np.linalg.lstsq(np.vstack([log_sizes, np.ones(len(log_sizes))]).T, log_counts, rcond=None)[0]

	fractal_dimension = -m
	return fractal_dimension

def plot_fractal_dimension(box_sizes, box_counts, fractal_dimension, imageName, type):
	plt.figure()
	plt.plot(np.log(box_sizes), np.log(box_counts), 'o-', label='Quantidade de Caixas')
	plt.xlabel('Log(tamanho da caixa)')
	plt.ylabel('Log(quantidade de caixas)')
	plt.grid(True)
	plt.title(f'Dimensão Fractal: {fractal_dimension}')
	plt.savefig(f'src/df/{type}/{imageName}h.png')

type = 'black'
imagens = ['00', '23', '48', '68', '75', '144']

with open(f'src/data/{type}/df.csv', mode='w', newline='') as f:
	writer = csv.writer(f)

	for imageName in imagens:
		image = cv2.cvtColor(cv2.imread(f'src/{type}/{imageName}h.jpg'), cv2.COLOR_RGB2GRAY)

		box_sizes = np.logspace(start = 1, stop = int(np.log2(np.min(image.shape)) - 1), num = int(np.log2(np.min(image.shape)) - 1), base = 2).astype(int)
		box_counts = box_counting(image, box_sizes)

		fractal_dimension = calculate_fractal_dimension(box_counts, box_sizes)
		writer.writerow([imageName, fractal_dimension])

		plot_fractal_dimension(box_sizes, box_counts, fractal_dimension, imageName, type)